# Interfacing sage and the LMFDB --- a prototype

The [lmfdb](http://www.lmfdb.org/) and [sagemath](http://www.sagemath.org/) are both great things, but they don't currently talk to each other. Much of the lmfdb calls sage, but the lmfdb also includes vast amounts of data on $L$-functions and modular forms (hence the name) that is not accessible from within sage.

This is an example prototype of an interface to the lmfdb from sage. Keep in mind that this is **a prototype** and every aspect can change. But we hope to show what may be possible in the future. If you have requests, comments, or questions, **please request/comment/ask** either now, or at my email: `david@lowryduda.com`.

> Note that this notebook is available on http://davidlowryduda.com or https://gist.github.com/davidlowryduda/deb1f88cc60b6e1243df8dd8f4601cde, and the code is available at https://github.com/davidlowryduda/sage2lmfdb

Let's dive into an example.

In [1]:
# These names will change
from sage.all import *
import LMFDB2sage.elliptic_curves as lmfdb_ecurve

In [2]:
lmfdb_ecurve.search(rank=1)

[Elliptic Curve defined by y^2 + x*y = x^3 - 887688*x - 321987008 over Rational Field,
 Elliptic Curve defined by y^2 + x*y + y = x^3 - x^2 + 10795*x - 97828 over Rational Field,
 Elliptic Curve defined by y^2 + x*y + y = x^3 - x^2 - 2294115305*x - 42292668425178 over Rational Field,
 Elliptic Curve defined by y^2 + x*y + y = x^3 - x^2 - 3170*x - 49318 over Rational Field,
 Elliptic Curve defined by y^2 + y = x^3 + 1050*x - 26469 over Rational Field,
 Elliptic Curve defined by y^2 + x*y = x^3 - x^2 - 1240542*x - 531472509 over Rational Field,
 Elliptic Curve defined by y^2 + y = x^3 - x^2 + 8100*x - 263219 over Rational Field,
 Elliptic Curve defined by y^2 + x*y = x^3 + 637*x - 68783 over Rational Field,
 Elliptic Curve defined by y^2 + y = x^3 + x^2 + 36*x - 380 over Rational Field,
 Elliptic Curve defined by y^2 + y = x^3 + x^2 - 2535*x - 49982 over Rational Field]

In [3]:
Es = lmfdb_ecurve.search(rank=2)
E = Es[0]
%time E.conductor()

CPU times: user 5 µs, sys: 0 ns, total: 5 µs
Wall time: 7.15 µs


11152

In [4]:
Es = lmfdb_ecurve.search(rank=1)
E = Es[0]
%time E.rank()

CPU times: user 36 µs, sys: 3 µs, total: 39 µs
Wall time: 44.8 µs


1

In [5]:
Es = lmfdb_ecurve.search(rank=2)
E = Es[0]
%time E.gens()

CPU times: user 14 µs, sys: 1 µs, total: 15 µs
Wall time: 17.2 µs


[(-1 : 1 : 1), (15 : 55 : 1)]

In [ ]:
E._json

In [ ]:
import time

In [ ]:
start = time.time()

for E in Es:
    print(E.rank())
    print(E.regulator())
    print(E.gens())

print("Duration: {} seconds.".format(time.time() - start))

In [ ]:

%time
for E in Es:
    print([E.gens(), E.rank(), E.regulator()])

In [ ]:
E = lmfdb_ecurve.search(rank=1)[0]
%time E.conductor()

This returns 10 elliptic curves of rank 1. But these are a bit different than sage's elliptic curves.

In [ ]:
Es = lmfdb_ecurve.search(rank=1)
E = Es[0]
print(type(E))

Note that the class of an elliptic curve is an lmfdb ElliptcCurve. But don't worry, this is a subclass of a normal elliptic curve. So we can call the normal things one might call on an elliptic curve.

In [ ]:
# Try autocompleting the following. It has all the things!
E.

![All the things](https://cdn.meme.am/cache/instances/folder251/500x/79798251/x-all-the-things-all-the-elliptic-curve-things.jpg)

This gives quick access to some data that is not stored within the LMFDB, but which is relatively quickly computable. For example,

In [ ]:
E.defining_ideal()

But one of the great powers is that there are some things which are computed and stored in the LMFDB, and not in sage. We can now immediately give many examples of rank 3 elliptic curves with:

In [6]:
Es = lmfdb_ecurve.search(conductor=11050, torsion_order=2)
print("There are {} curves returned.".format(len(Es)))
E = Es[0]
print(E)

There are 10 curves returned.
Elliptic Curve defined by y^2 + x*y = x^3 - 887688*x - 321987008 over Rational Field


And for these curves, the lmfdb contains data on its rank, generators, regulator, and so on.

In [ ]:
%time
print(E.gens())
print(E.rank())
print(E.regulator())

In [7]:
res = []
%time for E in Es: res.append([E.gens(), E.rank(), E.regulator()])

CPU times: user 548 µs, sys: 0 ns, total: 548 µs
Wall time: 412 µs


That's pretty fast, and this is because all of this was pulled from the LMFDB when the curves were returned by the `search()` function.

In this case, elliptic curves over the rationals are only an okay example, as they're really well studied and sage can compute much of the data very quickly. On the other hand, through the LMFDB there are millions of examples and corresponding data at one's fingertips.

### This is where we're really looking for input.

Think of what you might want to have easy access to through an interface from sage to the LMFDB, and tell us. We're actively seeking comments, suggestions, and requests. Elliptic curves over the rationals are a prototype, and the LMFDB has lots of (much more challenging to compute) data. There is data on the LMFDB that is simply not accessible from within sage.

**email: david@lowryduda.com, or post an issue on https://github.com/LMFDB/lmfdb/issues**



## Now let's describe what's going on under the hood a little bit

There is an API for the LMFDB at http://beta.lmfdb.org/api/. This API is a bit green, and we will change certain aspects of it to behave better in the future. A call to the API [looks like](http://beta.lmfdb.org/api/elliptic_curves/curves/?rank=i1&conductor=i11050&_format=json)

    http://beta.lmfdb.org/api/elliptic_curves/curves/?rank=i1&conductor=i11050
    
The result is a large mess of data, which can be exported as json and parsed.

But that's hard, and the resulting data are not sage objects. They are just strings or ints, and these require time *and thought* to parse.

So we created a module in sage that writes the API call and parses the output back into sage objects. The 22 curves given by the above API call are the same 22 curves returned by this call:

In [ ]:
Es = lmfdb_ecurve.search(rank=1, conductor=11050, max_items=25)
print(len(Es))
E = Es[0]

The total functionality of this search function is visible from its current documentation.

In [ ]:
# Execute this cell for the documentation
lmfdb_ecurve.search?

In [ ]:
E._json

In [ ]:
# So, for instance, one could perform the following search, finding a unique elliptic curve
lmfdb_ecurve.search(rank=2, torsion_order=3, degree=4608)

### What if there are no curves?

If there are no curves satisfying the search criteria, then a message is displayed and that's that. These searches may take a couple of seconds to complete.

For example, no elliptic curve in the database has rank 5.

In [ ]:
lmfdb_ecurve.search(rank=5)

### How does one step through the data?

Right now, at most 100 curves are returned in a single API call. This is the limit even from directly querying the API. But one can pass in the argument `base_item` (the name will probably change... to `skip`? or perhaps to `offset`?) to start returning at the `base_item`th element.

In [ ]:
from pprint import pprint
pprint(lmfdb_ecurve.search(rank=1, max_items=3))              # The last item in this list
print('')
pprint(lmfdb_ecurve.search(rank=1, max_items=3, base_item=2)) # should be the first item in this list

Included in the documentation is also a bit of hopefulness. Right now, the LMFDB API does not actually accept `max_conductor` or `min_conductor` (or arguments of that type). But it will sometime. (This introduces a few extra difficulties on the server side, and so it will take some extra time to decide how to do this).

In [ ]:
lmfdb_ecurve.search(rank=1, min_conductor=500, max_conductor=10000)  # Not implemented

Our `EllipticCurve_rational_field_lmfdb` class constructs a sage elliptic curve from the json and overrides (somem of the) the default methods in sage if there is quicker data available on the LMFDB. In principle, this new object is just a sage object with some slightly different methods.

Generically, documentation and introspection on objects from this class should work. Much of sage's documentation carries through directly.

In [ ]:
E.gens?

Modified methods should have a note indicating that the data comes from the LMFDB, and then give sage's documentation. This is not yet implemented. (So if you examine the current version, you can see some incomplete docstrings like `regulator()`.)

In [ ]:
E.regulator?

## This concludes our demo of an interface between sage and the LMFDB.

Thank you, and if you have any questions, comments, or concerns, please find me/email me/raise an issue on LMFDB's github.

![XKCD's automation](https://imgs.xkcd.com/comics/automation.png)